<a href="https://colab.research.google.com/github/plawanrath/pytorch_basics/blob/main/StanfordCars_Test_With_ChatGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# write a model in PyTorch that can given an input image predict the Car's year, make and model using the Stanford Cars dataset
# Preprocess the torchvision StanfordCars dataaset for this model
# create a training and test loop for this
# call the training and test loop for this model
# make a prediction of a car's year, make and model using this CarClassifier

import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets

# Define transformations to apply to the dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the dataset and apply the transformations
train_set = datasets.StanfordCars(root='./data', split='train', download=True, transform=transform)
test_set = datasets.StanfordCars(root='./data', split='test', download=True, transform=transform)

# Create data loaders to load the data in batches
train_loader = torch.utils.data.DataLoader(train_set, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=32, shuffle=False)

100%|██████████| 330960/330960 [00:01<00:00, 235653.46it/s]


Extracting data/stanford_cars/car_devkit.tgz to data/stanford_cars


  0%|          | 4063232/979269282 [00:03<13:53, 1170129.47it/s]


KeyboardInterrupt: ignored

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class CarClassifier(nn.Module):
    def __init__(self, num_classes):
        super(CarClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 28 * 28, 256)
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        x = x.view(-1, 64 * 28 * 28)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
# Define the training and test functions
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    return running_loss / len(train_loader.dataset)

def test(model, test_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            running_corrects += torch.sum(preds == labels.data)
    return running_loss / len(test_loader.dataset), running_corrects.double() / len(test_loader.dataset)

In [ ]:
# Define hyperparameters and other settings
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = 196
lr = 0.01
num_epochs = 5

In [ ]:
import torch.optim as optim
from tqdm.auto import tqdm


# Initialize the model, loss function, and optimizer
model = CarClassifier(num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

# Train the model
for epoch in tqdm(range(num_epochs)):
    train_loss = train(model, train_loader, criterion, optimizer, device)
    test_loss, test_acc = test(model, test_loader, criterion, device)
    print(f'Epoch {epoch+1}/{num_epochs} \nTrain Loss: {train_loss:.4f} \nTest Loss: {test_loss:.4f} Test Accuracy: {test_acc:.4f}\n')

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1/5 
Train Loss: 5.9292 
Test Loss: 5.2736 Test Accuracy: 0.0085

Epoch 2/5 
Train Loss: 5.2832 
Test Loss: 5.2731 Test Accuracy: 0.0085

Epoch 3/5 
Train Loss: 5.2830 
Test Loss: 5.2729 Test Accuracy: 0.0085

Epoch 4/5 
Train Loss: 5.2827 
Test Loss: 5.2730 Test Accuracy: 0.0085

Epoch 5/5 
Train Loss: 5.2828 
Test Loss: 5.2728 Test Accuracy: 0.0085

